In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# This header make the program uses proper number of GPUs.
# If not set this header, the program will use all GPUs.

import datetime
import numpy as np
import tensorflow as tf
import sys
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import model_from_json
import matplotlib.pyplot as plt
from IPython.display import clear_output
np.random.seed(123)  # for reproducibility

# Extra part for memory management
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
def Make_LSTM_result(state_condition, structure, peak_hour, pred_hour, LSTM_hour, 
                     num_unit, num_epoch, train_index, val_index, test_index, optmzr, loss_ftn):
    #file_condition = '1hPEAK_6hLSTM_log'
    
    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour
    
    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)
    
    file_condition = str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'
    
    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()
       
    allDataSet = read_csv('/home/kwyi/GOES_LSTM_csv/GOES15_pgt10_'+file_condition+'_log.csv', sep=',')
    allDataSet_size = len(allDataSet)
    
    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))
    
    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))
        
    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    #flux = allDataSet[['Year','Mon','Day','Hour','Min','b1','b2','b3','b4','b5','b6']].values
    flux = allDataSet[data_title].values
    pred_flux = allDataSet[pred_title].values
    now_flux = allDataSet['a0'].values
    now_Year = allDataSet['Year'].values
    now_Mon = allDataSet['Mon'].values
    now_Day = allDataSet['Day'].values
    now_Hour = allDataSet['Hour'].values
    now_Min = allDataSet['Min'].values
     
    d_flux = np.array([flux])
    d_pred_flux = np.array([pred_flux])
    d_now_flux = np.array([now_flux])
    d_now_Year = np.array([now_Year])
    d_now_Mon = np.array([now_Mon])
    d_now_Day = np.array([now_Day])
    d_now_Hour = np.array([now_Hour])
    d_now_Min = np.array([now_Min])
    
    
    #d_flux = np.array(d_flux).reshape(allDataSet_size, 1, 6)
    #inputFluxShape = (1,6)    
    d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    d_now_Year = np.array([d_now_Year]).reshape(allDataSet_size, 1)
    d_now_Mon = np.array([d_now_Mon]).reshape(allDataSet_size, 1)
    d_now_Day = np.array([d_now_Day]).reshape(allDataSet_size, 1)
    d_now_Hour = np.array([d_now_Hour]).reshape(allDataSet_size, 1)
    d_now_Min = np.array([d_now_Min]).reshape(allDataSet_size, 1)
    

    def M_LSTM_Stateful_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(len_PRED))
        model.summary()
        return model
            
    def M_LSTM_Stateless_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(len_PRED))
        model.summary()
        return model
    
    def M_LSTM_Stateful_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(7))
        model.add(Dense(len_PRED))
        model.summary()
        return model
            
    def M_LSTM_Stateless_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(7))
        model.add(Dense(len_PRED))
        model.summary()
        return model


    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();
            

    if structure is 'Inspace':
        weightfileName = '/home/kwyi/GOES_LSTM_TEST_WEIGHT/dupl_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
        testfileName = '/home/kwyi/GOES_LSTM_TEST_RESULT/dupl_'+state_condition+'_OnetimeOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
    else:
        if structure is 'SeaT':
            weightfileName = '/home/kwyi/GOES_LSTM_TEST_WEIGHT/dupl_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
            testfileName = '/home/kwyi/GOES_LSTM_TEST_RESULT/dupl_'+state_condition+'_OnetimeOutput_Dense7_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
        else:
            print("You entered wrong ")
            sys.exit()
            

    plot_losses = PlotLosses()
    
    if state_condition is 'Stateful':
        inputFluxShape = (1,1,len_LSTM)
        if structure is 'Inspace':
            model = M_LSTM_Stateful_Inspace(inputFluxShape)
        if structure is 'SeaT':
            model = M_LSTM_Stateful_SeaT(inputFluxShape)
    else:
        if state_condition is 'Stateless':
            inputFluxShape = (1,len_LSTM)
            if structure is 'Inspace':
                model = M_LSTM_Stateless_Inspace(inputFluxShape)
            if structure is 'SeaT':
                model = M_LSTM_Stateless_SeaT(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()
 

    val_index  = train_index + val_index
    test_index = val_index + test_index
    
    trainX = d_flux[:train_index]
    trainY = d_pred_flux[:train_index]
    trainYear = d_now_Year[:train_index]
    trainMon = d_now_Mon[:train_index]
    trainDay = d_now_Day[:train_index]
    trainHour = d_now_Hour[:train_index]
    trainMin = d_now_Min[:train_index]
    
    valX = d_flux[train_index:val_index]
    valY = d_pred_flux[train_index:val_index]
    valYear = d_now_Year[train_index:val_index]
    valMon = d_now_Mon[train_index:val_index]
    valDay = d_now_Day[train_index:val_index]
    valHour = d_now_Hour[train_index:val_index]
    valMin = d_now_Min[train_index:val_index]
    
    testX = d_flux[val_index:test_index]
    testY = d_pred_flux[val_index:test_index]
    testNowY = d_now_flux[val_index:test_index]
    testYear = d_now_Year[val_index:test_index]
    testMon = d_now_Mon[val_index:test_index]
    testDay = d_now_Day[val_index:test_index]
    testHour = d_now_Hour[val_index:test_index]
    testMin = d_now_Min[val_index:test_index]
    
    model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
    
    
    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
            model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, validation_data=(valX, valY))

        
    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
    

    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('Year' + ','+ 'Month' + ','+ 'Day' + ','+ 'Hour' + ','+ 'Min' + ',' + 'Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')
        
    for idx2 in range(0, len(testX)):
        
        if idx2 != len(testX)-1:

            NowY = testNowY[idx2:idx2+1]
     
            Year = testYear[idx2][0]
            Mon = testMon[idx2][0]
            Day = testDay[idx2][0]
            Hour = testHour[idx2][0]
            Min = testMin[idx2][0]
        
            Year2 = testYear[idx2+1][0]
            Mon2 = testMon[idx2+1][0]
            Day2 = testDay[idx2+1][0]
            Hour2 = testHour[idx2+1][0]
            Min2 = testMin[idx2+1][0]
        
            pre_time = datetime.datetime(Year, Mon, Day, Hour, Min)
            next_time = datetime.datetime(Year2, Mon2, Day2, Hour2, Min2)
            Second_Term = peak_hour*3600   
            
            Diff = next_time - pre_time
            Diff = Diff.total_seconds()
            Diff = Diff/Second_Term
            Diff = Diff-1
            
            pred = []
            pred_in = testX[idx2:idx2+1]  #    (1, 1, 96)        

            resultY = model.predict(pred_in)
      
            date = str(Year) + ',' + str(Mon) + ',' + str(Day) + ',' + str(Hour) + ',' + str(Min)

            f.write(date)
            f.write(',' + str(NowY[0][0]))
            
            for nwr in range(len_PRED):
                f.write(',' + str(resultY[0][nwr]))   
                
            f.write('\n')
            pred_in = []
            resultY = []
            print(idx2)
            
            if Diff >= 1:                            
                Term = datetime.timedelta(seconds=Second_Term)
                ddd = pre_time
                for n_diff in range(int(Diff)):    
                    ddd = ddd + Term
                    #print(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute) + '\n')
                    f.write(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute))
                    for wr in range(len_PRED+1):
                        f.write(',' + str(-99999))
                    f.write('\n')
                    
                    
        else:
            NowY = testNowY[idx2:idx2+1]
     
            Year = testYear[idx2][0]
            Mon = testMon[idx2][0]
            Day = testDay[idx2][0]
            Hour = testHour[idx2][0]
            Min = testMin[idx2][0]
            
            pred = []
            pred_in = testX[idx2:idx2+1]  #    (1, 1, 96)        

            resultY = model.predict(pred_in)
      
            date = str(Year) + ',' + str(Mon) + ',' + str(Day) + ',' + str(Hour) + ',' + str(Min)

            f.write(date)
            f.write(',' + str(NowY[0][0]))
            for nwr in range(len_PRED):
                f.write(',' + str(resultY[0][nwr]))   
            f.write('\n')
            pred_in = []
            resultY = []
            print(idx2)
        
    f.close()

###################################################################################################

In [ ]:
num_epoch = 4800
num_unit = 240

Make_LSTM_result('Stateless', 'Inspace', 1, 24, 48, num_unit, num_epoch, 42066, 8536, 8122, "Adam",'mean_squared_error')

#Make_LSTM_result('Stateless', 'Inspace', 1, 24, 72, num_unit, num_epoch, 41580, 8464, 7954, "Adam",'mean_squared_error')
#Make_LSTM_result('Stateless', 'SeaT', 1, 24, 96, num_unit, num_epoch, 41124, 8392, 7786, "Adam",'mean_squared_error')
#Make_LSTM_result('Stateless', 'SeaT', 1, 24, 48, num_unit, num_epoch, 42066, 8536, 8122, "Adam",'mean_squared_error')

allDataSet:  (58724, 77)
allDataSet_size:  58724
allDataSet Type:  <class 'pandas.core.frame.DataFrame'>
allDataSet_size Type:  <class 'int'>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 240)               277440    
_________________________________________________________________
dense_2 (Dense)              (None, 24)                5784      
Total params: 283,224
Trainable params: 283,224
Non-trainable params: 0
_________________________________________________________________
Train on 42066 samples, validate on 8536 samples
Epoch 1/4800
 - 1s - loss: 0.1878 - acc: 0.0436 - val_loss: 0.0131 - val_acc: 0.0446
Epoch 2/4800
 - 0s - loss: 0.0913 - acc: 0.0482 - val_loss: 0.0092 - val_acc: 0.0439
Epoch 3/4800
 - 0s - loss: 0.0725 - acc: 0.0545 - val_loss: 0.0091 - val_acc: 0.0457
Epoch 4/4800
 - 0s - loss: 0.0650 - acc: 0.0630 - val_loss: 0.0089 - val_acc: 0.0504
E

Epoch 88/4800
 - 0s - loss: 0.0490 - acc: 0.0713 - val_loss: 0.0087 - val_acc: 0.0489
Epoch 89/4800
 - 0s - loss: 0.0489 - acc: 0.0697 - val_loss: 0.0088 - val_acc: 0.0470
Epoch 90/4800
 - 0s - loss: 0.0487 - acc: 0.0708 - val_loss: 0.0092 - val_acc: 0.0507
Epoch 91/4800
 - 0s - loss: 0.0487 - acc: 0.0738 - val_loss: 0.0086 - val_acc: 0.0492
Epoch 92/4800
 - 0s - loss: 0.0492 - acc: 0.0707 - val_loss: 0.0086 - val_acc: 0.0476
Epoch 93/4800
 - 0s - loss: 0.0487 - acc: 0.0707 - val_loss: 0.0089 - val_acc: 0.0490
Epoch 94/4800
 - 0s - loss: 0.0487 - acc: 0.0728 - val_loss: 0.0089 - val_acc: 0.0505
Epoch 95/4800
 - 0s - loss: 0.0487 - acc: 0.0724 - val_loss: 0.0085 - val_acc: 0.0506
Epoch 96/4800
 - 0s - loss: 0.0485 - acc: 0.0734 - val_loss: 0.0094 - val_acc: 0.0471
Epoch 97/4800
 - 0s - loss: 0.0490 - acc: 0.0735 - val_loss: 0.0090 - val_acc: 0.0494
Epoch 98/4800
 - 0s - loss: 0.0485 - acc: 0.0712 - val_loss: 0.0091 - val_acc: 0.0535
Epoch 99/4800
 - 0s - loss: 0.0486 - acc: 0.0735 - val

Epoch 183/4800
 - 0s - loss: 0.0466 - acc: 0.0777 - val_loss: 0.0091 - val_acc: 0.0453
Epoch 184/4800
 - 0s - loss: 0.0465 - acc: 0.0769 - val_loss: 0.0087 - val_acc: 0.0474
Epoch 185/4800
 - 0s - loss: 0.0467 - acc: 0.0759 - val_loss: 0.0095 - val_acc: 0.0469
Epoch 186/4800
 - 0s - loss: 0.0466 - acc: 0.0764 - val_loss: 0.0091 - val_acc: 0.0494
Epoch 187/4800
 - 0s - loss: 0.0466 - acc: 0.0735 - val_loss: 0.0086 - val_acc: 0.0504
Epoch 188/4800
 - 0s - loss: 0.0465 - acc: 0.0759 - val_loss: 0.0094 - val_acc: 0.0470
Epoch 189/4800
 - 0s - loss: 0.0465 - acc: 0.0760 - val_loss: 0.0089 - val_acc: 0.0504
Epoch 190/4800
 - 0s - loss: 0.0464 - acc: 0.0765 - val_loss: 0.0087 - val_acc: 0.0466
Epoch 191/4800
 - 0s - loss: 0.0466 - acc: 0.0750 - val_loss: 0.0088 - val_acc: 0.0491
Epoch 192/4800
 - 0s - loss: 0.0464 - acc: 0.0733 - val_loss: 0.0087 - val_acc: 0.0497
Epoch 193/4800
 - 0s - loss: 0.0465 - acc: 0.0755 - val_loss: 0.0084 - val_acc: 0.0466
Epoch 194/4800
 - 0s - loss: 0.0469 - acc: 

 - 0s - loss: 0.0448 - acc: 0.0764 - val_loss: 0.0094 - val_acc: 0.0513
Epoch 278/4800
 - 0s - loss: 0.0446 - acc: 0.0780 - val_loss: 0.0084 - val_acc: 0.0501
Epoch 279/4800
 - 0s - loss: 0.0447 - acc: 0.0772 - val_loss: 0.0093 - val_acc: 0.0500
Epoch 280/4800
 - 0s - loss: 0.0446 - acc: 0.0795 - val_loss: 0.0082 - val_acc: 0.0497
Epoch 281/4800
 - 0s - loss: 0.0447 - acc: 0.0764 - val_loss: 0.0088 - val_acc: 0.0511
Epoch 282/4800
 - 0s - loss: 0.0446 - acc: 0.0781 - val_loss: 0.0083 - val_acc: 0.0521
Epoch 283/4800
 - 0s - loss: 0.0446 - acc: 0.0767 - val_loss: 0.0089 - val_acc: 0.0478
Epoch 284/4800
 - 0s - loss: 0.0446 - acc: 0.0773 - val_loss: 0.0086 - val_acc: 0.0499
Epoch 285/4800
 - 0s - loss: 0.0444 - acc: 0.0771 - val_loss: 0.0090 - val_acc: 0.0479
Epoch 286/4800
 - 0s - loss: 0.0444 - acc: 0.0778 - val_loss: 0.0098 - val_acc: 0.0499
Epoch 287/4800
 - 0s - loss: 0.0446 - acc: 0.0771 - val_loss: 0.0091 - val_acc: 0.0476
Epoch 288/4800
 - 0s - loss: 0.0445 - acc: 0.0783 - val_lo

Epoch 372/4800
 - 0s - loss: 0.0426 - acc: 0.0793 - val_loss: 0.0082 - val_acc: 0.0490
Epoch 373/4800
 - 0s - loss: 0.0426 - acc: 0.0768 - val_loss: 0.0085 - val_acc: 0.0489
Epoch 374/4800
 - 0s - loss: 0.0427 - acc: 0.0785 - val_loss: 0.0087 - val_acc: 0.0524
Epoch 375/4800
 - 0s - loss: 0.0429 - acc: 0.0746 - val_loss: 0.0085 - val_acc: 0.0466
Epoch 376/4800
 - 0s - loss: 0.0427 - acc: 0.0792 - val_loss: 0.0090 - val_acc: 0.0500
Epoch 377/4800
 - 0s - loss: 0.0427 - acc: 0.0797 - val_loss: 0.0100 - val_acc: 0.0469
Epoch 378/4800
 - 0s - loss: 0.0426 - acc: 0.0785 - val_loss: 0.0097 - val_acc: 0.0480
Epoch 379/4800
 - 0s - loss: 0.0427 - acc: 0.0789 - val_loss: 0.0084 - val_acc: 0.0505
Epoch 380/4800
 - 0s - loss: 0.0429 - acc: 0.0769 - val_loss: 0.0083 - val_acc: 0.0496
Epoch 381/4800
 - 0s - loss: 0.0427 - acc: 0.0777 - val_loss: 0.0089 - val_acc: 0.0486
Epoch 382/4800
 - 0s - loss: 0.0425 - acc: 0.0800 - val_loss: 0.0095 - val_acc: 0.0519
Epoch 383/4800
 - 0s - loss: 0.0424 - acc: 

...

Epoch 4025/4800
 - 0s - loss: 0.0249 - acc: 0.0910 - val_loss: 0.0128 - val_acc: 0.0458
Epoch 4026/4800
 - 0s - loss: 0.0250 - acc: 0.0911 - val_loss: 0.0135 - val_acc: 0.0469
Epoch 4027/4800
 - 0s - loss: 0.0249 - acc: 0.0930 - val_loss: 0.0131 - val_acc: 0.0444
Epoch 4028/4800
 - 0s - loss: 0.0249 - acc: 0.0930 - val_loss: 0.0145 - val_acc: 0.0485
Epoch 4029/4800
 - 0s - loss: 0.0248 - acc: 0.0928 - val_loss: 0.0124 - val_acc: 0.0456
Epoch 4030/4800
 - 0s - loss: 0.0246 - acc: 0.0935 - val_loss: 0.0158 - val_acc: 0.0469
Epoch 4031/4800
 - 0s - loss: 0.0250 - acc: 0.0914 - val_loss: 0.0128 - val_acc: 0.0450
Epoch 4032/4800
 - 0s - loss: 0.0249 - acc: 0.0922 - val_loss: 0.0122 - val_acc: 0.0451
Epoch 4033/4800
 - 0s - loss: 0.0249 - acc: 0.0920 - val_loss: 0.0125 - val_acc: 0.0449
Epoch 4034/4800
 - 0s - loss: 0.0248 - acc: 0.0913 - val_loss: 0.0125 - val_acc: 0.0450
Epoch 4035/4800
 - 0s - loss: 0.0248 - acc: 0.0894 - val_loss: 0.0135 - val_acc: 0.0446
Epoch 4036/4800
 - 0s - loss: 0.

 - 0s - loss: 0.0246 - acc: 0.0927 - val_loss: 0.0151 - val_acc: 0.0497
Epoch 4119/4800
 - 0s - loss: 0.0249 - acc: 0.0919 - val_loss: 0.0181 - val_acc: 0.0458
Epoch 4120/4800
 - 0s - loss: 0.0253 - acc: 0.0915 - val_loss: 0.0125 - val_acc: 0.0463
Epoch 4121/4800
 - 0s - loss: 0.0248 - acc: 0.0930 - val_loss: 0.0124 - val_acc: 0.0448
Epoch 4122/4800
 - 0s - loss: 0.0248 - acc: 0.0923 - val_loss: 0.0130 - val_acc: 0.0463
Epoch 4123/4800
 - 0s - loss: 0.0248 - acc: 0.0925 - val_loss: 0.0127 - val_acc: 0.0401
Epoch 4124/4800
 - 0s - loss: 0.0251 - acc: 0.0900 - val_loss: 0.0123 - val_acc: 0.0473
Epoch 4125/4800
 - 0s - loss: 0.0247 - acc: 0.0911 - val_loss: 0.0126 - val_acc: 0.0445
Epoch 4126/4800
 - 0s - loss: 0.0247 - acc: 0.0944 - val_loss: 0.0122 - val_acc: 0.0404
Epoch 4127/4800
 - 0s - loss: 0.0249 - acc: 0.0917 - val_loss: 0.0193 - val_acc: 0.0464
Epoch 4128/4800
 - 0s - loss: 0.0255 - acc: 0.0903 - val_loss: 0.0130 - val_acc: 0.0451
Epoch 4129/4800
 - 0s - loss: 0.0247 - acc: 0.09

Epoch 4212/4800
 - 0s - loss: 0.0245 - acc: 0.0921 - val_loss: 0.0134 - val_acc: 0.0452
Epoch 4213/4800
 - 0s - loss: 0.0247 - acc: 0.0935 - val_loss: 0.0127 - val_acc: 0.0456
Epoch 4214/4800
 - 0s - loss: 0.0247 - acc: 0.0926 - val_loss: 0.0123 - val_acc: 0.0435
Epoch 4215/4800
 - 0s - loss: 0.0247 - acc: 0.0921 - val_loss: 0.0126 - val_acc: 0.0432
Epoch 4216/4800
 - 0s - loss: 0.0248 - acc: 0.0934 - val_loss: 0.0131 - val_acc: 0.0465
Epoch 4217/4800
 - 0s - loss: 0.0248 - acc: 0.0920 - val_loss: 0.0143 - val_acc: 0.0444
Epoch 4218/4800
 - 0s - loss: 0.0248 - acc: 0.0909 - val_loss: 0.0126 - val_acc: 0.0423
Epoch 4219/4800
 - 0s - loss: 0.0251 - acc: 0.0923 - val_loss: 0.0158 - val_acc: 0.0484
Epoch 4220/4800
 - 0s - loss: 0.0248 - acc: 0.0926 - val_loss: 0.0129 - val_acc: 0.0472
Epoch 4221/4800
 - 0s - loss: 0.0247 - acc: 0.0913 - val_loss: 0.0129 - val_acc: 0.0456
Epoch 4222/4800
 - 0s - loss: 0.0249 - acc: 0.0926 - val_loss: 0.0145 - val_acc: 0.0474
Epoch 4223/4800
 - 0s - loss: 0.

 - 0s - loss: 0.0251 - acc: 0.0905 - val_loss: 0.0126 - val_acc: 0.0483
Epoch 4306/4800
 - 0s - loss: 0.0246 - acc: 0.0922 - val_loss: 0.0143 - val_acc: 0.0486
Epoch 4307/4800
 - 0s - loss: 0.0248 - acc: 0.0935 - val_loss: 0.0129 - val_acc: 0.0432
Epoch 4308/4800
 - 0s - loss: 0.0251 - acc: 0.0931 - val_loss: 0.0161 - val_acc: 0.0481
Epoch 4309/4800
 - 0s - loss: 0.0247 - acc: 0.0924 - val_loss: 0.0274 - val_acc: 0.0478
Epoch 4310/4800
 - 0s - loss: 0.0264 - acc: 0.0905 - val_loss: 0.0125 - val_acc: 0.0456
Epoch 4311/4800
 - 0s - loss: 0.0248 - acc: 0.0936 - val_loss: 0.0126 - val_acc: 0.0448
Epoch 4312/4800
 - 0s - loss: 0.0251 - acc: 0.0918 - val_loss: 0.0127 - val_acc: 0.0480
Epoch 4313/4800
 - 0s - loss: 0.0246 - acc: 0.0929 - val_loss: 0.0129 - val_acc: 0.0459
Epoch 4314/4800
 - 0s - loss: 0.0246 - acc: 0.0901 - val_loss: 0.0179 - val_acc: 0.0476
Epoch 4315/4800
 - 0s - loss: 0.0248 - acc: 0.0912 - val_loss: 0.0132 - val_acc: 0.0438
Epoch 4316/4800
 - 0s - loss: 0.0248 - acc: 0.09

Epoch 4399/4800
 - 0s - loss: 0.0244 - acc: 0.0924 - val_loss: 0.0130 - val_acc: 0.0428
Epoch 4400/4800
 - 0s - loss: 0.0248 - acc: 0.0922 - val_loss: 0.0188 - val_acc: 0.0483
Epoch 4401/4800
 - 0s - loss: 0.0250 - acc: 0.0911 - val_loss: 0.0133 - val_acc: 0.0469
Epoch 4402/4800
 - 0s - loss: 0.0246 - acc: 0.0924 - val_loss: 0.0128 - val_acc: 0.0459
Epoch 4403/4800
 - 0s - loss: 0.0244 - acc: 0.0926 - val_loss: 0.0128 - val_acc: 0.0437
Epoch 4404/4800
 - 0s - loss: 0.0248 - acc: 0.0902 - val_loss: 0.0131 - val_acc: 0.0430
Epoch 4405/4800
 - 0s - loss: 0.0248 - acc: 0.0909 - val_loss: 0.0127 - val_acc: 0.0428
Epoch 4406/4800
 - 0s - loss: 0.0245 - acc: 0.0934 - val_loss: 0.0141 - val_acc: 0.0455
Epoch 4407/4800
 - 0s - loss: 0.0249 - acc: 0.0904 - val_loss: 0.0131 - val_acc: 0.0467
Epoch 4408/4800
 - 0s - loss: 0.0246 - acc: 0.0936 - val_loss: 0.0143 - val_acc: 0.0478
Epoch 4409/4800
 - 0s - loss: 0.0249 - acc: 0.0897 - val_loss: 0.0130 - val_acc: 0.0444
Epoch 4410/4800
 - 0s - loss: 0.